In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import warnings
warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

import missingno as msno

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data_dir = os.getenv('HOME')+'/aiffel/LCA_FY'

data_2017_path = join(data_dir, 'LCA_FY_2017.csv')
data_2018_path = join(data_dir, 'LCA_FY_2018.csv')
data_2019_path = join(data_dir, 'LCA_FY_2019.csv')
# 코로나 이후
data_2020_path = join(data_dir, 'LCA_FY_2020.csv')
data_2021_path = join(data_dir, 'LCA_FY_2021.csv')
data_2022_path = join(data_dir, 'LCA_FY_2022.csv')

data_2017 = pd.read_csv(data_2017_path)
data_2018 = pd.read_csv(data_2018_path)
data_2019 = pd.read_csv(data_2019_path)
data_2020 = pd.read_csv(data_2020_path)
data_2021 = pd.read_csv(data_2021_path)
data_2022 = pd.read_csv(data_2022_path)

In [4]:
data_2019.rename(columns={'Wage_Rate_Of_Pay' : 'Prevailing_Wage'}, inplace= True)
data_2020 = data_2020.drop('Quarter', axis= 1)
data_2021 = data_2021.drop('Quarter', axis= 1)

In [5]:
pd.concat([data_2017,data_2018,data_2019], ignore_index = True)

,Visa_Class,Employer_Name,SOC_Title,Job_Title,Full_Time_Position,Worksite,Prevailing_Wage,Unit_Of_Pay,Employer_Location,Employer_Country,Case_Status
0,H-1B,DISCOVER PRODUCTS INC.,Computer Systems Analysts,ASSOCIATE DATA INTEGRATION,Y,"Riverwoods, Illinois",59197.0,Year,"Riverwoods, Illinois",United States Of America,CERTIFIED-WITHDRAWN
1,H-1B,DFS SERVICES LLC,Operations Research Analysts,SENIOR ASSOCIATE,Y,"Riverwoods, Illinois",49800.0,Year,"Riverwoods, Illinois",United States Of America,CERTIFIED-WITHDRAWN
2,H-1B,EASTBANC TECHNOLOGIES LLC,Computer Programmers,.NET SOFTWARE PROGRAMMER,Y,"Washington, District of Columbia",76502.0,Year,"Washington, District of Columbia",United States Of America,CERTIFIED-WITHDRAWN
3,H-1B,INFO SERVICES LLC,"Computer Occupations, All Other",PROJECT MANAGER,Y,"Jersey City, New Jersey",90376.0,Year,"Livonia, Michigan",United States Of America,WITHDRAWN
4,H-1B,BB&T CORPORATION,Credit Analysts,ASSOCIATE - ESOTERIC ASSET BACKED SECURITIES,Y,"New York, New York",116605.0,Year,"Wilson, North Carolina",United States Of America,CERTIFIED-WITHDRAWN
...,...,...,...,...,...,...,...,...,...,...,...
1943621,H-1B,AMERICAN FAMILY MUTUAL INSURANCE COMPANY,15-2031,DIGITAL TOUCHPOINTS ANALYST,Y,"Madison, Wisconsin",85500.0,Year,"Madison, Wisconsin",United States Of America,CERTIFIED
1943622,H-1B,CAPGEMINI AMERICA INC,NaN,APPLICATIONS CONSULTANT 2,Y,NaN,80800.0,Year,"Chicago, Illinois",United States Of America,DENIED
1943623,H-1B,CAPGEMINI AMERICA INC,NaN,BUSINESS SYSTEMS ANALYST 2,Y,NaN,74500.0,Year,"Chicago, Illinois",United States Of America,DENIED
1943624,H-1B,CAPGEMINI AMERICA INC,NaN,APPLICATIONS CONSULTANT 2,Y,NaN,75900.0,Year,"Chicago, Illinois",United States Of America,DENIED


In [6]:
pd.concat([data_2020,data_2021,data_2022], ignore_index = True)

,Visa_Class,Employer_Name,SOC_Title,Job_Title,Full_Time_Position,Worksite,Prevailing_Wage,Unit_Of_Pay,Employer_Location,Employer_Country,Case_Status
0,H-1B,"JO-ANN STORES, INC.","Computer Occupations, All Other","APPLICATION ENGINEER, OMS [15-1199.02]",Y,"Hudson, Ohio",95118.0,Year,"Hudson, Ohio",United States Of America,Certified
1,H-1B,DENKEN SOLUTIONS INC.,"Software Developers, Applications",BI DEVELOPER II,Y,"Brentwood, Tennessee",39.0,Hour,"Irvine, California",United States Of America,Certified
2,H-1B,"EPITEC, INC.",Mechanical Engineers,QUALITY ENGINEER,Y,"Dearborn, Michigan",39.0,Hour,"Southfield, Michigan",United States Of America,Certified
3,H-1B,"SYSTEMS TECHNOLOGY GROUP, INC.","Software Developers, Applications","SOFTWARE DEVELOPER, APPLICATIONS",Y,"Taylor, Michigan",53.0,Hour,"Troy, Michigan",United States Of America,Certified
4,H-1B,E-GIANTS TECHNOLOGIES LLC,"Computer Occupations, All Other",QUALITY ENGINEER LEVEL II,Y,"Blue Ash, Ohio",65333.0,Year,"Clive, Iowa",United States Of America,Certified
...,...,...,...,...,...,...,...,...,...,...,...
1526242,H-1B,"Love Health, Inc.","Software Developers, Systems Software",React Native Engineer,Y,"Miami, Florida",101026.0,Year,"Miami, Florida",United States Of America,Withdrawn
1526243,H-1B,HEALTH CARE SERVICE CORPORATION,Computer Systems Analysts,Sr. Systems Analyst,Y,"Chicago, Illinois",111197.0,Year,"Chicago, Illinois",United States Of America,Withdrawn
1526244,H-1B,"Lithia Motors, Inc.",Database Architects,Data Engineer I,Y,"Beaverton, Oregon",56534.0,Year,"Medford, Oregon",United States Of America,Withdrawn
1526245,H-1B,Amazon.com Services LLC,"Software Developers, Systems Software",Quality Assurance Engineer II,Y,"Austin, Texas",90750.0,Year,"Seattle, Washington",United States Of America,Withdrawn


In [7]:
temp=pd.concat([data_2017,data_2018,data_2019], ignore_index = True)
#코로나 이후
temp2=pd.concat([data_2020,data_2021,data_2022], ignore_index = True)

In [8]:
temp.isnull().sum()

Visa_Class                 0
Employer_Name             85
SOC_Title                 18
Job_Title                 11
Full_Time_Position        11
Worksite              581019
Prevailing_Wage           12
Unit_Of_Pay              111
Employer_Location        154
Employer_Country      124658
Case_Status                0
dtype: int64

In [9]:
temp2.isnull().sum()

Visa_Class               0
Employer_Name            7
SOC_Title                2
Job_Title                1
Full_Time_Position       0
Worksite                94
Prevailing_Wage       1961
Unit_Of_Pay           1961
Employer_Location      278
Employer_Country         0
Case_Status              0
dtype: int64

In [10]:
temp = temp.drop('Job_Title', axis= 1)
temp = temp.drop('Employer_Country',axis =1)
temp = temp.drop(temp[(temp['Employer_Name'].isnull()) | (temp['SOC_Title'].isnull()) | (temp['Full_Time_Position'].isnull()) | (temp['Prevailing_Wage'].isnull()) | (temp['Unit_Of_Pay'].isnull())].index)
temp = temp.drop(temp[temp['Employer_Location'].isnull()].index)
temp.isnull().sum()

Visa_Class                 0
Employer_Name              0
SOC_Title                  0
Full_Time_Position         0
Worksite              580943
Prevailing_Wage            0
Unit_Of_Pay                0
Employer_Location          0
Case_Status                0
dtype: int64

In [11]:
temp2 = temp2.drop('Job_Title', axis= 1)
temp2 = temp2.drop('Employer_Country',axis =1)
temp2 = temp2.drop(temp2[(temp2['Employer_Name'].isnull()) | (temp2['SOC_Title'].isnull()) | (temp2['Full_Time_Position'].isnull()) | (temp2['Prevailing_Wage'].isnull()) | (temp2['Unit_Of_Pay'].isnull())].index)
temp2 = temp2.drop(temp2[temp2['Employer_Location'].isnull()].index)
temp2.isnull().sum()

Visa_Class             0
Employer_Name          0
SOC_Title              0
Full_Time_Position     0
Worksite              93
Prevailing_Wage        0
Unit_Of_Pay            0
Employer_Location      0
Case_Status            0
dtype: int64

In [12]:
temp['Worksite']=temp['Worksite'].fillna(temp['Employer_Location'])

In [13]:
temp2['Worksite']=temp2['Worksite'].fillna(temp2['Employer_Location'])

In [14]:
temp.isnull().sum()

Visa_Class            0
Employer_Name         0
SOC_Title             0
Full_Time_Position    0
Worksite              0
Prevailing_Wage       0
Unit_Of_Pay           0
Employer_Location     0
Case_Status           0
dtype: int64

In [15]:
temp2.isnull().sum()

Visa_Class            0
Employer_Name         0
SOC_Title             0
Full_Time_Position    0
Worksite              0
Prevailing_Wage       0
Unit_Of_Pay           0
Employer_Location     0
Case_Status           0
dtype: int64